## Optimization Summer 2022

In [1]:
import numpy as np

# Given data for V and E
V = np.array([211.24813, 240.01760, 271.28726, 305.16127, 341.74381, 381.13906])
E = np.array([-78.58421, -78.63681, -78.65265, -78.64328, -78.61798, -78.58299])

# Initial conditions
E0 = -78.65265
B0 = 0.5
V0 = 271.28726
B0_prime = 0.5
x0 = np.array([E0, B0, V0, B0_prime])

# Function for the Murnaghan equation (equation 8.1)
def fun_E(V, x):
    E0, B0, V0, B0_prime = x
    term1 = (V / V0) ** (1 - B0_prime) / (B0_prime * (B0_prime - 1))
    term2 = V / (B0_prime * V0) - 1 / (B0_prime - 1)
    return E0 + B0 * V0 * (term1 + term2)

# Objective function to minimize
def fun_opt(E, V, x):
    E_V = fun_E(V, x)
    return np.sum((E_V - E) ** 2)

# Central difference gradient calculation
def calc_gradient(E, V, x, step=1e-5):
    grad = np.zeros_like(x)
    for i in range(len(x)):
        x_forward = x.copy()
        x_backward = x.copy()
        x_forward[i] += step
        x_backward[i] -= step
        grad[i] = (fun_opt(E, V, x_forward) - fun_opt(E, V, x_backward)) / (2 * step)
    return grad

# Backtracking line search to satisfy Wolfe conditions
def back_track(E, V, xk, pk, grad, fun, alpha=1.0, rho=0.5, gamma=1e-4):
    while fun(E, V, xk + alpha * pk) > fun(E, V, xk) + gamma * alpha * np.dot(grad, pk):
        alpha *= rho
    return alpha

# Initial step using steepest descent
p0 = -calc_gradient(E, V, x0)
alpha = back_track(E, V, x0, p0, calc_gradient(E, V, x0), fun_opt)
xk = x0 + alpha * p0
s0 = p0

# Iterative optimization using nonlinear conjugate gradient (Fletcher-Reeves)
tolerance = 1e-5
max_iter = 1000
counter = 0
residual = np.linalg.norm(calc_gradient(E, V, xk))

while residual > tolerance:
    print(f"Iteration {counter}, Norm of Gradient = {residual:.6f}")

    # Calculate new gradient and update direction using Fletcher-Reeves
    pk = -calc_gradient(E, V, xk)
    beta = np.dot(pk, pk) / np.dot(p0, p0)
    sk = pk + beta * s0

    # Line search using backtracking
    alpha = back_track(E, V, xk, sk, calc_gradient(E, V, xk), fun_opt)

    # Update variables
    xk = xk + alpha * sk
    p0 = pk
    s0 = sk
    residual = np.linalg.norm(pk)
    counter += 1

    if counter >= max_iter:
        print("MAX_ITER REACHED!")
        break

# Display results
print("---------------------------")
print(f" E0 = {xk[0]:.6f}")
print(f" B0 = {xk[1]:.6f}")
print(f" V0 = {xk[2]:.6f}")
print(f" B0_prime = {xk[3]:.6f}")


Iteration 0, Norm of Gradient = 412.472596
Iteration 1, Norm of Gradient = 412.472596
Iteration 2, Norm of Gradient = 365.010211
Iteration 3, Norm of Gradient = 191.200327
Iteration 4, Norm of Gradient = 159.279873
Iteration 5, Norm of Gradient = 139.025999
Iteration 6, Norm of Gradient = 103.474431
Iteration 7, Norm of Gradient = 3.332246
Iteration 8, Norm of Gradient = 2.146041
Iteration 9, Norm of Gradient = 0.523297
Iteration 10, Norm of Gradient = 0.304347
Iteration 11, Norm of Gradient = 0.473639
Iteration 12, Norm of Gradient = 0.856942
Iteration 13, Norm of Gradient = 0.594029
Iteration 14, Norm of Gradient = 0.228241
Iteration 15, Norm of Gradient = 0.033047
Iteration 16, Norm of Gradient = 0.017017
Iteration 17, Norm of Gradient = 0.013393
Iteration 18, Norm of Gradient = 0.012598
Iteration 19, Norm of Gradient = 0.006927
Iteration 20, Norm of Gradient = 0.009841
Iteration 21, Norm of Gradient = 0.014963
Iteration 22, Norm of Gradient = 0.003264
Iteration 23, Norm of Gradient